# Project 1
## The Net Wars: Attack of the Bots
### Advanced Machine Learning - MECD (2022/2023)

### Autores

- Duarte Meneses - 2019216949
- Patricia Costa - 2019213995

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tensorflow as tf
# Just to make plots look better
plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams['axes.grid'] = True
plt.style.use('fivethirtyeight')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['lines.linewidth'] = 3

|Class  |Label |
| ----- | ---- | 
|Normal	|0     |
|Dos	|1     |
|R2L	|2     |
|U2R	|3     |
|Probe	|4     |

In [76]:
PATH_TO_DATA = 'dataset/'

test_students = pd.read_csv(PATH_TO_DATA + 'test_students.csv')
SampleID = test_students['SampleID']

train_students = pd.read_csv(PATH_TO_DATA + 'train_students.csv')


In [77]:
test_students = test_students.drop('SampleID', axis=1)

### Check Nulls and DataType

In [78]:
print(test_students.info())
print(train_students.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44556 entries, 0 to 44555
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     44556 non-null  int64  
 1   protocol_type                44556 non-null  object 
 2   service                      44556 non-null  object 
 3   flag                         44556 non-null  object 
 4   src_bytes                    44556 non-null  int64  
 5   dst_bytes                    44556 non-null  int64  
 6   land                         44556 non-null  int64  
 7   wrong_fragment               44556 non-null  int64  
 8   urgent                       44556 non-null  int64  
 9   hot                          44556 non-null  int64  
 10  num_failed_logins            44556 non-null  int64  
 11  logged_in                    44556 non-null  int64  
 12  num_compromised              44556 non-null  int64  
 13  root_shell      

In [79]:
train_students.replace('normal', 0, inplace=True)
train_students.replace('Dos', 1, inplace=True)
train_students.replace('R2L', 2, inplace=True)
train_students.replace('U2R', 3, inplace=True)
train_students.replace('Probe', 4, inplace=True)


In [80]:
train_students['attack_type'].value_counts(normalize = True)

0    0.518964
1    0.358846
4    0.095545
2    0.025808
3    0.000837
Name: attack_type, dtype: float64

### Categorical

In [81]:
#Service não contêm o mesmo núemero de valores
train_students = train_students.drop('service', axis=1)
test_students = test_students.drop('service', axis=1)

train_students = pd.get_dummies(train_students, columns=['protocol_type'], drop_first=True)
#train_students = train_students.drop('service', axis=1)
#train_students = pd.get_dummies(train_students, columns=['service'], drop_first=True)
train_students= pd.get_dummies(train_students, columns=['flag'], drop_first=True)

test_students = pd.get_dummies(test_students, columns=['protocol_type'], drop_first=True)
#test_students = test_students.drop('service', axis=1)
#test_students = pd.get_dummies(test_students, columns=['service'], drop_first=True)
test_students= pd.get_dummies(test_students, columns=['flag'], drop_first=True)


In [82]:
y = train_students['attack_type']
X = train_students.drop(columns=['attack_type'], axis=1)

### Primeira tentativa de ANN

In [ ]:
oversample = SMOTE()
over_X, over_y = oversample.fit_resample(X, y)
over_X_train, over_X_test, over_y_train, over_y_test = train_test_split(over_X, over_y, test_size=0.2, stratify=over_y, random_state=42)

classifier = Sequential()
# Defining the Input layer and FIRST hidden layer,both are same!
# relu means Rectifier linear unit function
classifier.add(Dense(units=32, input_dim=50, kernel_initializer='uniform', activation='relu'))
 
#Defining the SECOND hidden layer, here we have not defined input because it is
# second layer and it will get input as the output of first hidden layer
classifier.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
 

classifier.add(Dense(units=5, kernel_initializer='uniform', activation='softmax'))
 
# Optimizer== the algorithm of SGG to keep updating weights
# loss== the loss function to measure the accuracy
# metrics== the way we will compare the accuracy after each step of SGD
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
 
# fitting the Neural Network on the training data
survivalANN_Model=classifier.fit(over_X_train, over_y_train, batch_size=10 , epochs=10, verbose=1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

classifier.evaluate(X_test, y_test)

In [ ]:
preds = classifier.predict(test_students)

In [ ]:
predict_test = np.argmax(preds, axis=1)

In [ ]:
submission = pd.DataFrame({'SampleID': SampleID, 'Class': predict_test})
print(submission)
submission.to_csv('submission_ann.csv', index=False)

### Com vários parâmetros

In [84]:
# Defining a function for finding best hyperparameters
def FunctionFindBestParams(X_train, y_train):
    
    # Defining the list of hyper parameters to try
    TrialNumber=0
    batch_size_list=[5, 10, 15, 20, 32]
    epoch_list=[5, 10, 15, 20, 32]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Batch', 'Epoch', 'Accuracy'])
    
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            # Creating the classifier ANN model
            classifier = Sequential()
            classifier.add(Dense(units=32, input_dim=50, kernel_initializer='uniform', activation='relu'))
            classifier.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
            classifier.add(Dense(units=5, kernel_initializer='uniform', activation='softmax'))
            classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
 
            
            survivalANN_Model=classifier.fit(X_train,y_train, batch_size=batch_size_trial , epochs=epochs_trial, verbose=1)
            # Fetching the accuracy of the training
            Accuracy = survivalANN_Model.history['accuracy'][-1]
            
            # printing the results of the current iteration
            print(" ")
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', Accuracy)
            
            print("---------------------------------------------")
            df = pd.DataFrame({'TrialNumber': [TrialNumber], 
                                'Batch': [batch_size_trial],
                                'Epoch': [epochs_trial],
                                'Accuracy': [Accuracy]})
            SearchResultsData = pd.concat([SearchResultsData, df], ignore_index=True)
    return(SearchResultsData)
 
###############################################
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

ResultsData=FunctionFindBestParams(X_train, y_train)

Epoch 1/5
16634/16634 [==============================] - 21s 1ms/step - loss: 103.4161 - accuracy: 0.9102
Epoch 2/5
16634/16634 [==============================] - 43s 3ms/step - loss: 3.5716 - accuracy: 0.9380
Epoch 3/5
16634/16634 [==============================] - 22s 1ms/step - loss: 0.5725 - accuracy: 0.9428
Epoch 4/5
16634/16634 [==============================] - 43s 3ms/step - loss: 10.2281 - accuracy: 0.9490
Epoch 5/5
16634/16634 [==============================] - 26s 2ms/step - loss: 5.6808 - accuracy: 0.9504
 
1 Parameters: batch_size: 5 - epochs: 5 Accuracy: 0.9503535032272339
---------------------------------------------
Epoch 1/10
16634/16634 [==============================] - 35s 2ms/step - loss: 37.3429 - accuracy: 0.9204
Epoch 2/10
16634/16634 [==============================] - 28s 2ms/step - loss: 86.0273 - accuracy: 0.9420
Epoch 3/10
16634/16634 [==============================] - 26s 2ms/step - loss: 0.7632 - accuracy: 0.9463
Epoch 4/10
16634/16634 [===================

In [85]:
print(ResultsData)

   TrialNumber Batch Epoch  Accuracy
0            1     5     5  0.950354
1            2     5    10    0.9555
2            3     5    15  0.957977
3            4     5    20  0.958963
4            5     5    32  0.959432
5            6    10     5  0.952097
6            7    10    10   0.96376
7            8    10    15   0.95847
8            9    10    20  0.957243
9           10    10    32  0.961644
10          11    15     5  0.956594
11          12    15    10  0.952181
12          13    15    15  0.942406
13          14    15    20  0.962017
14          15    15    32  0.957604
15          16    20     5  0.950618
16          17    20    10  0.957892
17          18    20    15  0.956438
18          19    20    20  0.900755
19          20    20    32  0.951147
20          21    32     5  0.957063
21          22    32    10  0.955271
22          23    32    15  0.942959
23          24    32    20  0.969327
24          25    32    32  0.917733


In [87]:
maxi = ResultsData[ResultsData.Accuracy == ResultsData.Accuracy.max()]
maxi

,TrialNumber,Batch,Epoch,Accuracy
23,24,32,20,0.969327


In [ ]:
count = y.value_counts()

tam = len(y)

class_weight = {0: (count[3]/tam)*100, 1: (count[2]/tam)*100, 2: (count[1]/tam)*100, 3: (count[0]/tam)*100, 4: (count[4]/tam)*100}

class_weight

In [ ]:
'''
oversample = SMOTE()
over_X, over_y = oversample.fit_resample(X, y)
over_X_train, over_X_test, over_y_train, over_y_test = train_test_split(over_X, over_y, test_size=0.2, stratify=over_y, random_state=42)
'''


classifier = Sequential()

classifier.add(Dense(units=32, input_dim=50, kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
 

classifier.add(Dense(units=5, kernel_initializer='uniform', activation='softmax'))

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
 

survivalANN_Model=classifier.fit(over_X_train, over_y_train, batch_size=maxi.Batch , epochs=maxi.Epoch, verbose=1, class_weight=class_weight)

In [ ]:
classifier.evaluate(X_test, y_test)

pred = classifier.predict(X_test)

from sklearn.metrics import classification_report

print(classification_report(y_test, pred))
print('=========================================')


In [ ]:
preds = classifier.predict(test_students)
predict_test = np.argmax(preds, axis=1)

In [ ]:
submission = pd.DataFrame({'SampleID': SampleID, 'Class': predict_test})
print(submission)
submission.to_csv('submission_ann_weights.csv', index=False)